In [184]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [11]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,1], inplace = True)
labelsTrain.columns = [ "face_shape" , "img_name"]
labelsTrain = labelsTrain.astype({'face_shape': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,1], inplace = True)
labelsTest.columns = [ "face_shape" , "img_name"]
labelsTest = labelsTest.astype({'face_shape': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [133]:
#left to right 19 pixel cut 19%
av_lr=[]
for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(100, 100), interpolation=cv.INTER_CUBIC)
    lr=[]
    for j in range(0,100):
        
        for j2 in range(0,100):
            if image[j,j2] != 255:
                lr.append(j2)
                break
    
    av_lr.append(min(lr))

print('minimum_value:' + str(min(av_lr)))
print('mean_value:' + str(np.mean(av_lr)))
print('standard_deviation_value:' + str(np.std(av_lr)))
print('Z_score_value:' + str( (np.mean(av_lr)-min(av_lr))/np.std(av_lr) ) )

minimum_value:10
mean_value:23.3855
standard_deviation_value:4.914253732765535
Z_score_value:2.7238113308543404


In [139]:
#right to left 19 pixel cut 19%
av_lr=[]
for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(100, 100), interpolation=cv.INTER_CUBIC)
    lr=[]
    for j in range(0,100):
        
        for j2 in range(99,0,-1):
            if image[j,j2] != 255:
                lr.append(100-j2)
                break
    
    av_lr.append(min(lr))

print('minimum_value:' + str(min(av_lr)))
print('mean_value:' + str(np.mean(av_lr)))
print('standard_deviation_value:' + str(np.std(av_lr)))
print('Z_score_value:' + str( (np.mean(av_lr)-min(av_lr))/np.std(av_lr) ) )

minimum_value:12
mean_value:23.4961
standard_deviation_value:5.009249922892648
Z_score_value:2.2949743328760577


In [142]:
#up to down 15 pixel cut 15%
av_lr=[]
for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(100, 100), interpolation=cv.INTER_CUBIC)
    lr=[]
    for j in range(0,100):
        
        for j2 in range(0,100):
            if image[j2,j] != 255:
                lr.append(j2)
                break
    
    av_lr.append(min(lr))

print('minimum_value:' + str(min(av_lr)))
print('mean_value:' + str(np.mean(av_lr)))
print('standard_deviation_value:' + str(np.std(av_lr)))
print('Z_score_value:' + str( (np.mean(av_lr)-min(av_lr))/np.std(av_lr) ) )

minimum_value:1
mean_value:18.6578
standard_deviation_value:3.9393018620055
Z_score_value:4.482469386342079


In [143]:
#down to up 16 pixel 16%
av_lr=[]
for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(100, 100), interpolation=cv.INTER_CUBIC)
    lr=[]
    for j in range(0,100):
        
        for j2 in range(99,0,-1):
            if image[j2,j] != 255:
                lr.append(100-j2)
                break
    
    av_lr.append(min(lr))

print('minimum_value:' + str(min(av_lr)))
print('mean_value:' + str(np.mean(av_lr)))
print('standard_deviation_value:' + str(np.std(av_lr)))
print('Z_score_value:' + str( (np.mean(av_lr)-min(av_lr))/np.std(av_lr) ) )

minimum_value:9
mean_value:20.2186
standard_deviation_value:4.127373746100539
Z_score_value:2.718096467662787


In [183]:
#Load training set################################
imagesTrain = np.zeros((10000, 173, 155)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[75:420,95:405] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(155 , 173), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
imagesTrain = imagesTrain.reshape(10000,26815)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 173, 155))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[75:420,95:405]
    image = cv.resize(image, dsize=(155 , 173), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    
imagesTest = imagesTest.reshape(2500,26815)   
imagesTest = pd.DataFrame(imagesTest) 

2

In [226]:
#accuracy = []
#r=range(0,1000,100) #peaks at 950/1000
#for i in r:
model = KNeighborsClassifier(n_neighbors=1000)
model.fit(imagesTrain,labelsTrain['face_shape'])
model.score(imagesTest,labelsTest['face_shape'])
#plt.plot(r,accuracy)    

0.8236